In [30]:
import numpy as np
import pandas as pd
import os
import requests
import bs4
import json
import seaborn as sns
import re
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import project04 as proj

In [2]:
url = 'https://genius.com{}'

In [3]:
response = requests.get(url.format('/artists/Denzel-curry'))
soup = bs4.BeautifulSoup(response.content)

In [4]:
for i in soup.find_all('a', attrs={'class': 'vertical_album_card'}):
    print(i['title'])

Melt My Eyez, See Your Future
13LOOD 1N + 13LOOD OUT MIXX
ZUU
TA13OO
TA13OO: Act 2 (Gray)
TA13OO: Act 1 (Light)


In [5]:
new_url = soup.find_all('a', attrs={'class': 'full_width_button'})[1]['href']
new_url

'/artists/albums?for_artist_page=16690&id=Denzel-curry'

In [6]:
albums_request = requests.get(url.format(new_url))
albums = bs4.BeautifulSoup(albums_request.content)

In [7]:
album_lst = albums.find('ul', attrs={'class': 'album_list primary_list'}).find_all('li')
album_lst

[<li><a class="album_link" href="/albums/Denzel-curry/Ta13oo-act-1-light">TA13OO: Act 1 (Light)</a></li>,
 <li><a class="album_link" href="/albums/Denzel-curry/Imperial-streaming-version">Imperial (Streaming Version)</a></li>,
 <li><a class="album_link" href="/albums/Denzel-curry/Melt-my-eyez-see-your-future">Melt My Eyez, See Your Future</a></li>,
 <li><a class="album_link" href="/albums/Denzel-curry/Nostalgic-64">Nostalgic 64</a></li>,
 <li><a class="album_link" href="/albums/Denzel-curry/Zuu">ZUU</a></li>,
 <li><a class="album_link" href="/albums/Denzel-curry/Ta13oo">TA13OO</a></li>,
 <li><a class="album_link" href="/albums/Denzel-curry/Curry-wuz-here">Curry Wuz Here</a></li>,
 <li><a class="album_link" href="/albums/Denzel-curry/Imperial">Imperial</a></li>,
 <li><a class="album_link" href="/albums/Denzel-curry/King-remembered-underground-tape-19911995">King Remembered (Underground Tape 1991–1995)</a></li>,
 <li><a class="album_link" href="/albums/Denzel-curry/32-zel">32 Zel</a></li

In [8]:
song_lst = []
for i in album_lst:
    link = i.find('a')['href']
    new_url = url.format(link)
    response = requests.get(new_url)
    soup = bs4.BeautifulSoup(response.content)
    tracklist = soup.find_all('div', attrs={'class': 'column_layout-column_span column_layout-column_span--primary'})[1]
    for j in tracklist.find_all('div', attrs={'class': 'chart_row-content'}):
        link = j.find('a')['href']
        song_link = requests.get(link)
        song = bs4.BeautifulSoup(song_link.content)
        lyrics = ''
        for k in song.find_all('div', attrs={'class': 'Lyrics__Container-sc-1ynbvzw-7 dVtOne'}):
            lyrics = lyrics + k.text
        lyrics = re.sub(r'([a-z])([A-Z])', r'\1. \2', lyrics)
        song_lst.append(lyrics)

In [44]:
cv = CountVectorizer()
word_count = cv.fit_transform(song_lst)

In [10]:
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(), columns=['idf_weights'])

In [25]:
tfidf_vector = tfidf_transformer.transform(word_count)
first_song = tfidf_vector[40]

In [26]:
df = pd.DataFrame(first_song.T.todense(), index=cv.get_feature_names(), columns=['tfidf'])

In [27]:
df.sort_values(by=['tfidf'], ascending=False).head(10)

,tfidf
fresh,0.440562
glitter,0.284520
somethin,0.237535
bein,0.237535
the,0.228277
teeth,0.213434
trill,0.201992
pocket,0.201992
full,0.166449
so,0.157137


In [32]:
songs = '\x03 '.join(song_lst)

In [36]:
tokens = proj.tokenize(songs)

In [57]:
unigram = proj.NGramLM(2, tuple(tokens))

In [62]:
samp = unigram.sample(200)

In [63]:
samp

"\x02 main and lows on the liars that green so now it ' t have it float , acting like Pistol Pete . Let it a black on my shlong , and . All of the Tec 9 Trey . That nigga here bruh , and they hoppin ' t mind at a bitch a metaphor for the street ' cause you bout to get hella pain stop ? Ooh ) Let me be coolin . A . Sky is with it float . Do things gon ' m saying , yuh . Increase with my lowest , no peace to fire nigga in my diamond you wasn ' s no fuck around bangin ' t have faith , bitch , girl you wanna be the day ) Pockets too culo ( Yuh , rest home . I keep the torturer . Yeah ) Put me on our rank on yourself down on that ' m timid and my head like a Taliban with the whole globe . I shine . And looking just another young nigga , that I never leave my brain , I got me won ' on my residence . Say , Valentine red . while your"